In [1]:
#Lowercase
def lowercase(text):
    return text.lower()

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

from underthesea import word_tokenize

def tokenize(text):
    return word_tokenize(text, format="text")

# import urllib
# # Download vietnamese stop words
# data=urllib.request.urlopen('https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt').read()
data = open("vietnamese-stopwords-dash.txt", "r", encoding="utf8").read()

# Read stop words
stopwords= [ x for x in data.splitlines() ]
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

import re
def remove_html_url(text):
    return re.sub(r'http\S+', '', text)

import emoji

def remove_emoji(text):
    def get_emoji_regexp():
        # Sort emoji by length to make sure multi-character emojis are
    # matched first
        emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
        pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
        return re.compile(pattern)

    exp = get_emoji_regexp()
    return exp.sub(u'', text)

def preprocess_text(text):
    # '''Input : String, output : String'''
      text_url = remove_html_url(text)
      text_emoji = remove_emoji(text_url)
    
      text_lowercase = lowercase(text_emoji)
      text_punctuation = remove_punctuation(text_lowercase)
      
      text_tokenize = tokenize(text_punctuation)
      text_stopwords = remove_stopwords(text_tokenize)
  
      return text_stopwords

In [2]:
import pandas as pd
data = pd.read_csv('tgdd_product_description_grouped.csv')

In [3]:
data.sample(5)

,name,description
82,/dtdd/vivo-y02t,Điện thoại vivo này có thiết kế đơn giản nhưng...
5,/dtdd/iphone-14-pro-256gb,"Phong cách thiết kế hiện đại, tối giản vẫn đượ..."
61,/dtdd/samsung-galaxy-a05s-6gb,Galaxy A05s 6GB lấy cảm hứng thiết kế từ các p...
17,/dtdd/masstel-lux-20,Chiếc điện thoại Masstel này có kích thước bề ...
24,/dtdd/nokia-105-4g,Toàn thân của máy được hoàn thiện từ nhựa với ...


In [4]:
data['description'] = data['description'].apply(preprocess_text)

In [15]:
data.sample(5)

,name,description
73,/dtdd/samsung-galaxy-s23-ultra,thiết_kế samsung galaxy s23 ultra thừa_hưởng k...
17,/dtdd/masstel-lux-20,điện_thoại masstel kích_thước bề ngang 538 mm ...
42,/dtdd/oppo-reno10-pro,thiết_kế reno10 pro 5 g trông ấn_tượng kiểu th...
68,/dtdd/samsung-galaxy-a34-5g-128gb,thiết_kế đẹp hiện_đại galaxy a34 5 g có_mặt lư...
14,/dtdd/masstel-izi-10,masstel izi 10 4 g thiết_kế điện_thoại phổ_thô...


In [20]:
dataDescription = list(data['description'].values)

In [21]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

In [22]:
## kiem tra ngon ngu
vi = 0
en = 0
remove_index = []
for i in range(len(dataDescription)):
  try:
    if detect(dataDescription[i]) == 'vi':
      vi+=1
    else : 
      en+=1
      remove_index.append(i)
  except:
    remove_index.append(i)
    continue

In [23]:
## loai bo cot ngon ngu
data_old = dataDescription
dataDescription = []
for i in range(len(data_old)):
  if i in remove_index:
    continue
  else : dataDescription.append(data_old[i])

In [24]:
data['description'] = dataDescription

In [28]:
data.sample(5)

,name,description
53,/dtdd/realme-c30s-3gb-64gb,realme c30s máy thiết_kế đẹp nổi_bật tầm_giá n...
46,/dtdd/oppo-reno11-pro,trải_nghiệm điện_thoại oppo reno11 pro thiết_k...
69,/dtdd/samsung-galaxy-a54-5g-128gb,2 tuần trải_nghiệm galaxy a54 ngạc_nhiên khả_n...
10,/dtdd/itel-it9010,itel it9010 khung mặt lưng hoàn_thiện nhựa thi...
107,/may-tinh-bang/ipad-pro-m2-12-9-inch,tổng_quan tablet máy hoàn_thiện mặt lưng kim_l...


In [26]:
data.to_csv('tgdd_product_description_grouped_preprocessed.csv', index=False)